In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, Tool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.retrievers import WikipediaRetriever
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini-2024-07-18"
)


def save_file(docs, title):
    with open(f"./{title}.txt", "w", encoding="utf-8") as file:
        file.write(str(docs))



def wiki_search(topic):
    retriever = WikipediaRetriever(top_k_results=3)
    docs = retriever.get_relevant_documents(topic)
    return docs


class SearchArgsSchema(BaseModel):
    term: str = Field(description="The term you will search for")


class WikipediaSearchTool(BaseTool):
    name="WikipediaSearchTool"
    description = """
    A tool getting results from Wikipedia by using the given keyword.
    """

    args_schema = SearchArgsSchema

    def _run(self, term):
        wiki_result = wiki_search(term)
        save_file(wiki_result,"wiki")
        return wiki_result   


class DDGSearchTool(BaseTool):
    name="DDGSearchTool"
    description = """
    A tool getting results from DuckDuckGo by using the given keyword.
    """

    args_schema = SearchArgsSchema

    def _run(self, term):
        ddg = DuckDuckGoSearchAPIWrapper()
        ddg_result =  ddg.run(term)
        save_file(ddg_result, "ddg")
        return ddg_result



agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DDGSearchTool(),
    ],
)

prompt = "Research about the XZ backdoor"

agent.run(prompt)




> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `XZ backdoor`


[Document(page_content='In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.\nWhile xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.\n\n\n== Background ==\nMicrosoft employee and PostgreSQL developer And

'The **XZ backdoor** refers to a malicious backdoor that was introduced into the Linux utility **xz**, specifically within the **liblzma** library, in versions **5.6.0** and **5.6.1**. This backdoor was discovered in **February 2024** and is attributed to an account using the name **"Jia Tan"**. The backdoor allows an attacker who possesses a specific **Ed448 private key** to execute remote code on affected Linux systems. It has been assigned the **CVE** number **CVE-2024-3094** and has a **CVSS score of 10.0**, indicating a critical vulnerability.\n\n### Discovery and Mechanism\nThe backdoor was uncovered by software developer **Andres Freund** while investigating performance issues in **Debian Sid**. Freund noticed unusual CPU usage during SSH connections and errors in **Valgrind**, a memory debugging tool. Upon further investigation, he reported his findings to the **Openwall Project\'s** security mailing list, which alerted various software vendors.\n\nThe backdoor operates by alte